In [2]:
import mysql.connector
import pandas as pd


In [2]:
%pip install mysql-connector-python pandas


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 2.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:

# Configurações de conexão com o MySQL no Docker
db_config = {
    'host': '192.168.137.4',  # Endereço do MySQL no Docker (localhost ou o IP do contêiner), no caso do meu notebook é 192.168.137.4
    'port': 3306,         # Porta do MySQL
    'user': 'root',       # Usuário do MySQL
    'password': 'rootpassword',  # Senha configurada no Docker
    'database': 'N3'      # Nome do banco de dados
}

# Conectar ao banco de dados MySQL
connection = mysql.connector.connect(**db_config)

# Consulta SQL
query = "SELECT * FROM consumo_energia_eletrica;"

# Usar pandas para ler a consulta diretamente em um DataFrame
df = pd.read_sql(query, connection)

# Fechar a conexão
connection.close()

# Exibir o DataFrame
print(df)


/tmp/ipykernel_11386/474337244.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


        ano  mes sigla_uf tipo_consumo  numero_consumidores  consumo
0      2004    1       TO        Total                  0.0    65876
1      2004    1       BA        Total                  0.0  1444451
2      2004    1       PR        Total                  0.0  1596274
3      2004    1       RS        Total                  0.0  1780912
4      2004    1       GO        Total                  0.0   630624
...     ...  ...      ...          ...                  ...      ...
38875  2023   12       BA  Residencial            5932740.0   718461
38876  2023   12       PA  Residencial            2614106.0   478726
38877  2023   12       AC  Residencial             245699.0    58301
38878  2023   12       RN  Residencial            1390080.0   210119
38879  2023   12       SC  Residencial            2719791.0   655389

[38880 rows x 6 columns]
